In [25]:
!pip install pandas geopandas matplotlib seaborn plotly h3


In [49]:
import pandas as pd

# Load the CSV (update path if needed)
df = pd.read_csv("911.csv")

# Show structure
print(df.columns)
df.head()


Index(['lat', 'lng', 'desc', 'zip', 'title', 'timeStamp', 'twp', 'addr', 'e'], dtype='object')


,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1


In [50]:
# Convert timestamp
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

# Extract date-related features
df['hour'] = df['timeStamp'].dt.hour
df['dayofweek'] = df['timeStamp'].dt.dayofweek
df['date'] = df['timeStamp'].dt.date

# Simplify title to high-level call type
# Handle potential non-string values in 'title' column
df['call_type'] = df['title'].apply(lambda x: x.split(':')[0].strip() if isinstance(x, str) else str(x))

# Drop any rows with missing coordinates
df = df.dropna(subset=['lat', 'lng'])

df.head()

,lat,lng,desc,zip,title,timeStamp,twp,addr,e,hour,dayofweek,date,call_type
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1,17,3,2015-12-10,EMS
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,17,3,2015-12-10,EMS
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1,14,3,2015-12-10,Fire
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1,16,3,2015-12-10,EMS
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1,16,3,2015-12-10,EMS


In [28]:
df.shape


(663522, 13)

In [51]:
import pandas as pd

# Assuming your DataFrame is named 'df'

# Missing Values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)


Missing Values:
 lat              0
lng              0
desc             0
zip          80199
title            0
timeStamp        0
twp            293
addr             0
e                0
hour             0
dayofweek        0
date             0
call_type        0
dtype: int64


In [52]:
df['zip'].fillna('Unknown', inplace=True)


<ipython-input-52-a455045261e1>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['zip'].fillna('Unknown', inplace=True)
<ipython-input-52-a455045261e1>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['zip'].fillna('Unknown', inplace=True)


In [53]:
df['title'].fillna('No title', inplace=True)


<ipython-input-53-fb6d14fe299e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['title'].fillna('No title', inplace=True)


In [54]:
df['zip'].fillna('Unknown', inplace=True)
df['title'].fillna('No title', inplace=True)
df['timeStamp'].fillna(df['timeStamp'].max(), inplace=True)
df['twp'].fillna(df['twp'].mode()[0], inplace=True)
df['addr'].fillna('No address', inplace=True)
df['e'].fillna(df['e'].mode()[0], inplace=True)
df['hour'].fillna(df['hour'].median(), inplace=True)
df['dayofweek'].fillna(df['dayofweek'].mode()[0], inplace=True)


<ipython-input-54-c45ce1550614>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['title'].fillna('No title', inplace=True)
<ipython-input-54-c45ce1550614>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

In [55]:
# Convert 'date' column to datetime objects, handling errors
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Fill NaN values with the most recent date
df['date'].fillna(df['date'].max(), inplace=True)

<ipython-input-55-189369f41a2f>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['date'].fillna(df['date'].max(), inplace=True)


In [56]:
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)


Missing Values:
 lat          0
lng          0
desc         0
zip          0
title        0
timeStamp    0
twp          0
addr         0
e            0
hour         0
dayofweek    0
date         0
call_type    0
dtype: int64


In [ ]:

# Unique Values
for column in df.columns:
    unique_values = df[column].unique()
    print(f"\nUnique values for {column}:\n", unique_values)

In [57]:
df.shape

(663522, 13)

In [58]:
# Create a dictionary that maps 'twp' values to zip codes
# You can customize this mapping or automate it if you have external data

# First, check the unique 'twp' values in your DataFrame
unique_twp_values = df['twp'].unique()

# Example: Manually map 'twp' values to zip codes based on known region-to-zipcode logic
zip_mapping = {  # Assign zip code for Region B
    # Add more mappings here based on your data
}

# If you want to automatically map all unique 'twp' values, you can assign default zip codes
# Example: Creating a default mapping where each 'twp' gets a unique zip
default_zip_mapping = {twp: f'ZIP{index+10000}' for index, twp in enumerate(unique_twp_values)}

# Optionally, use the manual or default mapping
zip_mapping = default_zip_mapping  # Use the default zip mapping

# Now, replace 'Unknown' in the 'zip' column based on 'twp'
df['zip'] = df.apply(lambda row: zip_mapping.get(row['twp'], row['zip']) if row['zip'] == 'Unknown' else row['zip'], axis=1)

# Print the updated DataFrame to see the changes
print(df.head())


         lat        lng                                               desc  \
0  40.297876 -75.581294  REINDEER CT & DEAD END;  NEW HANOVER; Station ...   
1  40.258061 -75.264680  BRIAR PATH & WHITEMARSH LN;  HATFIELD TOWNSHIP...   
2  40.121182 -75.351975  HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...   
3  40.116153 -75.343513  AIRY ST & SWEDE ST;  NORRISTOWN; Station 308A;...   
4  40.251492 -75.603350  CHERRYWOOD CT & DEAD END;  LOWER POTTSGROVE; S...   

        zip                    title           timeStamp                twp  \
0   19525.0   EMS: BACK PAINS/INJURY 2015-12-10 17:10:52        NEW HANOVER   
1   19446.0  EMS: DIABETIC EMERGENCY 2015-12-10 17:29:21  HATFIELD TOWNSHIP   
2   19401.0      Fire: GAS-ODOR/LEAK 2015-12-10 14:39:21         NORRISTOWN   
3   19401.0   EMS: CARDIAC EMERGENCY 2015-12-10 16:47:36         NORRISTOWN   
4  ZIP10003           EMS: DIZZINESS 2015-12-10 16:56:52   LOWER POTTSGROVE   

                         addr  e  hour  dayofweek       

In [37]:

df.shape

(663522, 13)

In [61]:
print(df.columns.tolist())

['lat', 'lng', 'desc', 'zip', 'title', 'timeStamp', 'twp', 'addr', 'e', 'hour', 'dayofweek', 'date', 'call_type', 'h3_cell']


In [59]:
import h3
print(h3.__version__)


4.2.2


In [60]:
import h3

def latlon_to_h3(row, resolution=7):
    return h3.latlng_to_cell(row['lat'], row['lng'], resolution)

df['h3_cell'] = df.apply(latlon_to_h3, axis=1)


In [62]:
df.to_csv('911_cleaned.csv', index=False)


In [63]:
df_cleaned = pd.read_csv('911_cleaned.csv')
df_cleaned.head()
df_cleaned.shape

(663522, 14)

In [66]:
# 🛠️ Create the datetime column
df_cleaned['ds'] = pd.to_datetime(df_cleaned['date'] + ' ' + df_cleaned['hour'].astype(str) + ':00:00')

# ✅ Now you can group by 'h3_cell' and 'ds'
df_grouped = df_cleaned.groupby(['h3_cell', 'ds']).size().reset_index(name='calls_count')

# Rename for NeuralForecast
df_nf = df_grouped.rename(columns={'h3_cell': 'unique_id', 'calls_count': 'y'})

# Final check
print(df_nf.head())


         unique_id                  ds  y
0  87195d2e0ffffff 2018-07-03 19:00:00  1
1  87195d2e0ffffff 2018-11-03 12:00:00  1
2  8726ee941ffffff 2018-07-20 06:00:00  1
3  872912649ffffff 2018-05-17 06:00:00  1
4  872912649ffffff 2018-06-11 00:00:00  2


In [67]:
!pip install neuralforecast


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [68]:
from neuralforecast.models import NHITS
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss


In [70]:
# One-hot encode call types (e.g., EMS, Fire, Traffic)
df_onehot = pd.get_dummies(df_cleaned[['h3_cell', 'ds', 'call_type']], columns=['call_type'])

# Group by location and time
df_grouped = df_onehot.groupby(['h3_cell', 'ds']).sum().reset_index()

df_grouped.head()


,h3_cell,ds,call_type_EMS,call_type_Fire,call_type_Traffic
0,87195d2e0ffffff,2018-07-03 19:00:00,0,1,0
1,87195d2e0ffffff,2018-11-03 12:00:00,0,1,0
2,8726ee941ffffff,2018-07-20 06:00:00,0,1,0
3,872912649ffffff,2018-05-17 06:00:00,1,0,0
4,872912649ffffff,2018-06-11 00:00:00,1,1,0


In [71]:
# Melt to long format (required by NeuralForecast / TFT-style input)
df_long = df_grouped.melt(id_vars=['h3_cell', 'ds'],
                          var_name='call_type',
                          value_name='y')

# Rename columns
df_long.rename(columns={'h3_cell': 'unique_id'}, inplace=True)

df_long.head()


,unique_id,ds,call_type,y
0,87195d2e0ffffff,2018-07-03 19:00:00,call_type_EMS,0
1,87195d2e0ffffff,2018-11-03 12:00:00,call_type_EMS,0
2,8726ee941ffffff,2018-07-20 06:00:00,call_type_EMS,0
3,872912649ffffff,2018-05-17 06:00:00,call_type_EMS,1
4,872912649ffffff,2018-06-11 00:00:00,call_type_EMS,1


In [72]:
df_long['series_id'] = df_long['unique_id'] + '_' + df_long['call_type']
df_model = df_long[['series_id', 'ds', 'y']]
df_model.rename(columns={'series_id': 'unique_id'}, inplace=True)


<ipython-input-72-4c9c0bc6addd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.rename(columns={'series_id': 'unique_id'}, inplace=True)


In [74]:
from neuralforecast.models import TFT
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss


In [ ]:
H = 24

tft_model = TFT(
    h=H,
    input_size=24*7,
    max_steps=1500,
    loss=MQLoss(level=[90])
)

nf = NeuralForecast(models=[tft_model], freq='H')
nf.fit(df_model)

forecast = nf.predict()


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MQLoss                   | 3      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 512    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 613 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K  | train
6 | output_adapter          | Linear                   | 387    | train
-------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]